# Segmentació d’Imatges amb U-Net

En aquesta pràctica treballarem amb U-Net, una de les arquitectures més utilitzades en tasques de segmentació d'imatges. L'objectiu principal és consolidar els coneixements teòrics adquirits, implementant i experimentant amb aquesta xarxa utilitzant un conjunt de dades artificial i simple.

La segmentació d'imatges és una tasca fonamental en visió per computador, ja que permet classificar cada píxel d'una imatge en una o més categories. En aquesta pràctica, el nostre objectiu serà entrenar una xarxa U-Net perquè pugui identificar regions específiques dins d’imatges generades artificialment. Aquest enfocament simplificat permetrà concentrar-nos en els aspectes clau de la implementació i el funcionament del model, sense les complicacions que podrien sorgir amb conjunts de dades més complexos.

El conjunt de dades que utilitzarem estarà format per imatges sintètiques que contenen formes geomètriques bàsiques (com cercles, quadrats o creus), amb les seves corresponents màscares que indiquen les àrees que cal segmentar. Això ens permetrà obtenir resultats visuals ràpidament i entendre millor com la xarxa aprèn a identificar patrons específics.

L'arquitectura que emprarem es pot utilitzar, per problemes més complexes, però amb les limitacions temporals que tenim no es recomenable.

In [ ]:
from collections import OrderedDict
from glob import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import torch
import torch.nn as nn
import torch.optim as optim
from IPython import display
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm.auto import tqdm

## Dataset

Emprarem un dataset sintètic. Aquest originalment s'emprava per dur a terme tasques de Intel·ligència Artificial Explicable (XAI). Nosaltres ho emprarem per dur a terme una tasca de segmentació. Podeu trobar el dataset complet al següent [enllaç](https://github.com/miquelmn/aixi-dataset/releases/tag/1.5.0).

![Imatges dataset](https://ars.els-cdn.com/content/image/1-s2.0-S0004370224001152-gr001.jpg)

L'objectiu amb aquest dataset es segmentar del fons les formes geomètriques, emprant una U-Net. Nosaltres i a causa de les limitacions temporals que tenim i de recursos emprarem una versió [reduida](https://github.com/miquelmn/aa_2526/releases/tag/pr9).

# Dataset de segmentació

A l’hora de treballar amb tasques de segmentació d’imatges, és essencial tenir un conjunt de dades que inclogui tant les imatges d’entrada com les màscares que representen les etiquetes de segmentació. Un dataset per segmentació ha de contenir:

1. Imatges d’entrada: Aquestes són les imatges que el model utilitzarà per aprendre. Poden estar en formats com JPEG o PNG.
2. Màscares de segmentació: Cada màscara és una imatge on cada píxel té un valor que representa la classe a la qual pertany (per exemple, 0 per fons, 1 per objecte). Les màscares han de tenir la mateixa mida que les imatges d’entrada.

In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import numpy as np
from torchvision import transforms

class Formes(Dataset):
    """
    TXUKI segmentation dataset.
    """
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_filenames = [f for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

        if self.transform is None:
            self.image_to_tensor = transforms.ToTensor()
            self.mask_to_tensor = transforms.Compose([
                transforms.ToTensor(),
                transforms.Lambda(lambda x: x.squeeze().long())
            ])


    def __len__(self):
        return len(self.img_filenames)


    def __getitem__(self, idx):
        img_filename = self.img_filenames[idx]
        mask_filename = img_filename

        img_path = os.path.join(self.img_dir, img_filename)
        mask_path = os.path.join(self.mask_dir, mask_filename)

        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        if self.transform:
            image = self.transform(image)
        if self.transform is None:
            image = self.image_to_tensor(image)
            mask = self.mask_to_tensor(mask)
        else:
             image = transforms.ToTensor()(image)
             mask_tensor = transforms.ToTensor()(mask)
             mask = mask_tensor.squeeze().long()

        return image, mask

Per instanciar aquest tipus de dataset es similar a com ho hem fet fins ara. **Per tal de simplificar l'entrenament només emprarem 500 mostres per l'entrenament i 200 de test**.

In [ ]:
PATH_DADES = "./mini"  # POSA EL TEU!
# Dades entrenament
path_train = f"{PATH_DADES}/train"

img_files = sorted(glob(path_train + "/image/*.png"))
label_files = sorted(glob(path_train + "/mask/*.png"))
img_files = img_files[:500]
label_files = label_files[:500]

print("total training images", len(img_files))

# Dades validacio

path_val = f"{PATH_DADES}/val"
img_files_val = sorted(glob(path_val + "/image/*.png"))
label_files_val = sorted(glob(path_val + "/mask/*.png"))
img_files_val = img_files_val[:200]
label_files_val = label_files_val[:200]

print("total test images", len(img_files_val))

In [ ]:
train_batch_size = 4
test_batch_size = 4

# Definim una seqüència (composició) de transformacions
transform = transforms.Compose([
    transforms.ToTensor(),
    ## TODO: Put if necessary
])

train_data = Formes(img_files, label_files, transform)
val_data = Formes(img_files_val, label_files_val, transform)

train_loader = torch.utils.data.DataLoader(train_data, train_batch_size)
val_loader = torch.utils.data.DataLoader(val_data, test_batch_size)

## Entrenament
L’entrenament d’una xarxa U-Net consisteix a ensenyar al model a predir les màscares de segmentació corresponents a les imatges d’entrada. Aquest procés implica ajustar els pesos de la xarxa per minimitzar l’error entre les prediccions i les etiquetes reals (màscares). En aquesta secció, configurarem tot el necessari per entrenar el model amb el conjunt de dades generat prèviament.

Hi ha tot un conjunt de peculiaritats que fan que aquest entrenament difereixi respecte els vists fins ara:
- **Funció de pèrdua**. El Dice Coefficient és una mesura d’avaluació utilitzada en segmentació d’imatges per comparar la superposició entre la màscara predita i la màscara real. El seu valor oscil·la entre 0 (cap coincidència) i 1 (coincidència perfecta), i es calcula com el doble de la intersecció entre les dues màscares dividit per la seva suma total. També es pot emprar ``BCE``.
- **Sortida de la xarxa**. El tipus de problema determina el nombre de canals de sortida de la U-Net: per segmentació binària, la sortida és un únic canal amb valors que representen la probabilitat de pertànyer a la classe positiva (fons o objecte). En canvi, per segmentació multiclasse, la sortida té un canal per a cada classe i els valors representen la probabilitat de cada píxel de pertànyer a cadascuna de les classes.
- **Funcions d'activació**. Les funcions d'activació a la sortida d'una U-Net depenen del tipus de segmentació: en segmentació binària, s'utilitza una sigmoid per comprimir els valors entre 0 i 1, representant la probabilitat de pertànyer a la classe positiva. En segmentació, **sense superposició**, multiclasse, s'aplica una softmax per convertir els valors de cada canal en probabilitats normalitzades, assegurant que la suma sigui 1 per píxel.

### El model

In [ ]:
class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        #TODO


    def forward(self, x):
        #TODO


## La funció de pèrdua

Per fer tasques de segmentació, una de les funcions de pèrdua que podem emprar és el _Diceloss_ (intersecció vs unió):  El coeficient de _Dice_ s'utilitza habitualment en tasques de segmentació d'imatges com a mesura de la superposició entre les màscares de segmentació entre la predicció i el _ground truth_. El  _Diceloss_, és el complementari del coeficient de _Dice_, es pot utilitzar com a funció de pèrdua per entrenar models per a tasques de segmentació.

Dice Coefficient $= 2 \times \frac{|X \cap Y|}{|X| + |Y|}$



On:

- $X$ és la màscara de segmentació prevista.
- $Y$ és la màscara de segmentació de la veritat del sòl.
- $∣⋅∣$ denota la cardinalitat o el nombre d'elements d'un conjunt.

In [ ]:
class DiceLoss(nn.Module):

    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 0.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
                y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

### Bucle d'entrenament

El bucle d'entrenament és un poc diferent al vist fins ara. En particular, en aquests moments els gràfics es van actualitzant a mesura que aprèn i així podem tenir un idea de cm va l'entrenament.

In [ ]:
#TODO

# Tasques a fer

1. Implementa la U-Net per detectar les formes geomètriques.
2. Empra com a funció de pèrdua tant la ``Dice`` com la ``BCE``, quina diferència hi ha?
3. Fer un nou entrenament per segmentar de forma separada els diferents tipus de figura. Has d'adaptar el model. Quins canvis faries?